# Cadence :

- Look at the output of the OpSim Database

## Requirements
- sqlite
- setup the sims stack before running the notebook (if not at least have healpy installed)
- setup an environment variable OPSIMDBPATH to point to the absolute path of the OpSim Database (unzipped)

## The Observing Strategy White Paper
- website: https://github.com/LSSTScienceCollaborations/ObservingStrategy
- Think about the requirements for different Science Cases.
- Suggest an Experiment : (for example, Jeonghee Rho) https://github.com/LSSTScienceCollaborations/ObservingStrategy/blob/master/opsim/README.md 

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import sqlite3
from sqlalchemy import create_engine

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
opsimdbpath = os.environ.get('OPSIMDBPATH')

In [5]:
print(opsimdbpath)

minion_1016_sqlite.db


In [6]:
engine = create_engine('sqlite:///' + opsimdbpath)

In [7]:
conn = sqlite3.connect(opsimdbpath)

In [8]:
cursor = conn.cursor()

In [ ]:
query = 'SELECT COUNT(*) FROM Summary'
cursor.execute(query)

In [ ]:
cursor.fetchall()

In [ ]:
opsimdf = pd.read_sql_query('SELECT * FROM Summary WHERE night < 1000', engine)

In [ ]:
opsimdf.head()

In [ ]:
# Definitions of the columns are 
opsimdf[['obsHistID', 'filter', 'night', 'expMJD',
         'fieldID', 'fieldRA', 'ditheredRA', 'ditheredRA', 'ditheredDec',
         'propID', 'fiveSigmaDepth']].head()

In [ ]:
opsimdf.propID.unique()

In [ ]:
xx = opsimdf.query('fieldID == 316')

In [ ]:
xx.head()

## Some unexpected issues

In [ ]:
xx.query('propID == 54')

## How to read the table:

- obsHistID indexes a pointing ('fieldRA', 'fieldDec', 'ditheredRA', 'ditheredDec')
- Additionally a pointing may be assigned a propID to describe what a pointing achieves
- The meaning of the propID is given in the Proposal Table. For minion_1016_sqlite.db, the WFD is 54, and the DDF is 56, but this coding might change.
- If a pointing achieves the task of succeeding in two different proposals, this is represented by haveing two records with the same pointng and different `propID`

In [ ]:
test = opsimdf.drop_duplicates()

In [ ]:
all(test == opsimdf)

In [ ]:
test = opsimdf.drop_duplicates(subset='obsHistID')

In [ ]:
len(test) == len(opsimdf)

In [ ]:
opsimdf.obsHistID.size

In [ ]:
opsimdf.obsHistID.unique().size

In [ ]:
test.obsHistID.size

### Using OpSimSummary

In [ ]:
import opsimsummary as oss

In [ ]:
# Read in the combined summary. 

In [ ]:
opsimout = oss.OpSimOutput.fromOpSimDB(opsimdbpath)

In [ ]:
help(oss.OpSimOutput)

In [ ]:
opsimDeep = oss.OpSimOutput.fromOpSimDB(opsimdbpath, subset='DDF')

In [ ]:
oss.OpSimOutput.get_allowed_subsets()

In [ ]:
odeep = oss.summarize_opsim.SummaryOpsim(opsimDeep.summary)

##  Recover some familiar numbers from this

In [ ]:
xx = opsimout.summary.groupby('fieldID').expMJD.agg('count')

In [ ]:
fig, ax = plt.subplots()
xx.hist(histtype='step', bins=np.arange(0., 5000, 50.), ax=ax)
ax.set_xlabel('fieldID')
ax.set_ylabel('Number of visits to field during survey')

In [ ]:
#DDF 
fig, ax = plt.subplots()
xx.hist(bins=np.arange(15000, 25000, 50), histtype='step')
ax.set_xlabel('fieldID')
ax.set_ylabel('Number of visits to field during survey DDF')

In [ ]:
xx[xx > 5000]

In [ ]:
# 1000 visits just in terms of exposure is 9 hrs 25 mins
fig, ax = plt.subplots()
xx = opsimout.summary.groupby(['night']).expMJD.agg('count')
xx.hist(histtype='step', bins=np.arange(0., 5000, 100.), ax=ax)
ax.set_xlabel('Number of visits in a night')